# Set up environment

In [13]:
import math
import numpy as np

from sklearn.datasets import load_digits, make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report
from sklearn.neural_network import MLPClassifier

import nn
from optim import SGDOptimizer

from supervised_learning import MyMLPClassifier
from dataset.load_data import sklearn_to_df, prepare_data_loader
from dataset.make_data import load_planar_dataset

# 1. Binary classification with simple data
(make classification sklearn)

In [14]:
def prepare_data():
    X, Y = make_classification(
        n_samples=1200,
        n_features=5,
        n_classes=2,
        n_clusters_per_class=1,
        n_redundant=0,
        n_informative=5,
        random_state=42
    )
    # X, Y = load_planar_dataset()
    X_tr, X_te, y_tr, y_te = train_test_split(X, Y, test_size=0.33, random_state=42)
    
    print ("train_x's shape: " + str(X_tr.shape))
    print ("test_x's shape: " + str(X_te.shape))
    print ("train_y's shape: " + str(y_tr.shape))
    print ("test_y's shape: " + str(y_te.shape))
    print()

    return X_tr, X_te, y_tr, y_te


def prepare_trainer(model):
    optimizer = SGDOptimizer(model, learning_rate=0.2, regularization=0.015)
    loss_func = nn.CrossEntropyLoss()
    return optimizer, loss_func

def build_model(n_in, n_class):
    np.random.seed(101)
    model = MyMLPClassifier(n_input=n_in, hiddens=[10], n_output=n_class, activation='relu')   
    return model

In [15]:
def run_my_mlp_for_binary_classification1():
    X_tr, X_te, y_tr, y_te = prepare_data()
    
    n_in, n_class = X_tr.shape[1], 2

    num_epochs = 200
    batch_size = 32

    model = build_model(n_in, n_class)
    optimizer, loss_func = prepare_trainer(model)

    model.info()

    # Training loop
    for epoch in range(num_epochs):
        # Prepare the data loader for training data
        data_loader = prepare_data_loader(X_tr, y_tr, batch_size)
        
        # Initialize counters for tracking training progress
        step = 0
        total_loss, total_correct = 0, 0
        total_sample = 0

        for batch_X, batch_y in data_loader:
            # forward pass: compute logits and loss
            batch_logit = model.forward(batch_X)        # output model: logit
            loss = loss_func.forward(batch_logit, batch_y)

            # backward pass and an optimization step
            optimizer.zero_grad()
            dout = loss_func.backward()
            model.backward(dout)
            optimizer.step()

            # log training progress
            step += 1
            total_loss += loss
            batch_yp = np.argmax(batch_logit, axis=1)   # logit --> label
            total_correct += np.sum(batch_yp == batch_y)
            total_sample += len(batch_y)
        print(f"Epoch: {epoch}, loss={total_loss / total_sample:.4f}, train_acc={total_correct / total_sample:.4f}")

    model.eval()

    ypred = np.argmax(model.forward(X_te), axis=1)
    print("\n My model: Classification report:\n", classification_report(y_te, ypred))

    
    skmodel = MLPClassifier()
    skmodel.fit(X_tr, y_tr)

     # Print information about the trained scikit-learn MLPClassifier
    print("\nScikit-learn MLPClassifier Info:")
    print("Number of layers:", skmodel.n_layers_)
    print("Number of neurons in each layer:", skmodel.hidden_layer_sizes)
    print("Number of output classes:", skmodel.n_outputs_)
    print("Activation function:", skmodel.activation)
    print("Solver:", skmodel.solver)
    print("Learning rate:", skmodel.learning_rate)
    print("Initial learning rate:", skmodel.learning_rate_init)
    print("Batch size:", skmodel.batch_size)
    print("Maximum number of iterations:", skmodel.max_iter)
    # Add more model-specific information as needed

    ypred = skmodel.predict(X_te)
    print("\n Sklearn model: Classification report:\n", classification_report(y_te, ypred))

In [16]:
run_my_mlp_for_binary_classification1()

train_x's shape: (804, 5)
test_x's shape: (396, 5)
train_y's shape: (804,)
test_y's shape: (396,)

MyMLPClassifier(
(linear): Linear(in_features=5, out_features=10, bias=True)
(relu): ReLU()
(linear): Linear(in_features=10, out_features=2, bias=True)
)
Epoch: 0, loss=0.6092, train_acc=0.7201
Epoch: 1, loss=0.4614, train_acc=0.8246
Epoch: 2, loss=0.4141, train_acc=0.8433
Epoch: 3, loss=0.3841, train_acc=0.8570
Epoch: 4, loss=0.3624, train_acc=0.8669
Epoch: 5, loss=0.3459, train_acc=0.8744
Epoch: 6, loss=0.3328, train_acc=0.8843
Epoch: 7, loss=0.3220, train_acc=0.8843
Epoch: 8, loss=0.3132, train_acc=0.8893
Epoch: 9, loss=0.3058, train_acc=0.8930
Epoch: 10, loss=0.2996, train_acc=0.8980
Epoch: 11, loss=0.2943, train_acc=0.8980
Epoch: 12, loss=0.2897, train_acc=0.9005
Epoch: 13, loss=0.2856, train_acc=0.9030
Epoch: 14, loss=0.2820, train_acc=0.9030
Epoch: 15, loss=0.2787, train_acc=0.9030
Epoch: 16, loss=0.2758, train_acc=0.9030
Epoch: 17, loss=0.2732, train_acc=0.9030
Epoch: 18, loss=0.2

/home/funny/miniconda3/envs/pytorch-env/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


# 2. Binary classification with planar data 
(Use Cross Entropty Loss - CE loss)

In [17]:
def prepare_data():
    X, Y = load_planar_dataset()
    X_tr, X_te, y_tr, y_te = train_test_split(X, Y, test_size=0.33, random_state=42)
    
    print ("train_x's shape: " + str(X_tr.shape))
    print ("test_x's shape: " + str(X_te.shape))
    print ("train_y's shape: " + str(y_tr.shape))
    print ("test_y's shape: " + str(y_te.shape))
    print()
    
    return X_tr, X_te, y_tr, y_te


def prepare_trainer(model):
    optimizer = SGDOptimizer(model, learning_rate=0.1, regularization=0.03, decay_learning_rate=False)
    loss_func = nn.CrossEntropyLoss()
    return optimizer, loss_func

def build_model(n_in, n_class):
    np.random.seed(101)
    model = MyMLPClassifier(n_input=n_in, hiddens=[4], n_output=n_class, activation='tanh')   
    return model

In [18]:
def run_my_mlp_for_binary_classification2():
    X_tr, X_te, y_tr, y_te = prepare_data()
    
    n_in, n_class = X_tr.shape[1], 2

    num_epochs = 480
    batch_size = 32

    model = build_model(n_in, n_class)
    optimizer, loss_func = prepare_trainer(model)

    model.info()

    # Training loop
    for epoch in range(num_epochs):
        # Prepare the data loader for training data
        data_loader = prepare_data_loader(X_tr, y_tr, batch_size)
        
        # Initialize counters for tracking training progress
        step = 0
        total_loss, total_correct = 0, 0
        total_sample = 0

        for batch_X, batch_y in data_loader:
            # forward pass: compute logits and loss
            batch_logit = model.forward(batch_X)        # output model: logit
            loss = loss_func.forward(batch_logit, batch_y)

            # backward pass and an optimization step
            optimizer.zero_grad()
            dout = loss_func.backward()
            model.backward(dout)
            optimizer.step()

            # log training progress
            step += 1
            total_loss += loss
            batch_yp = np.argmax(batch_logit, axis=1)   # logit --> label
            total_correct += np.sum(batch_yp == batch_y)
            total_sample += len(batch_y)
        print(f"Epoch: {epoch}, loss={total_loss / total_sample:.4f}, train_acc={total_correct / total_sample:.4f}")

    model.eval()

    ypred = np.argmax(model.forward(X_te), axis=1)
    print("\n My model: Classification report:\n", classification_report(y_te, ypred))

    
    skmodel = MLPClassifier()
    skmodel.fit(X_tr, y_tr)

     # Print information about the trained scikit-learn MLPClassifier
    print("\nScikit-learn MLPClassifier Info:")
    print("Number of layers:", skmodel.n_layers_)
    print("Number of neurons in each layer:", skmodel.hidden_layer_sizes)
    print("Number of output classes:", skmodel.n_outputs_)
    print("Activation function:", skmodel.activation)
    print("Solver:", skmodel.solver)
    print("Learning rate:", skmodel.learning_rate)
    print("Initial learning rate:", skmodel.learning_rate_init)
    print("Batch size:", skmodel.batch_size)
    print("Maximum number of iterations:", skmodel.max_iter)
    # Add more model-specific information as needed

    ypred = skmodel.predict(X_te)
    print("\n Sklearn model: Classification report:\n", classification_report(y_te, ypred))

In [19]:
model = run_my_mlp_for_binary_classification2()

train_x's shape: (670, 2)
test_x's shape: (330, 2)
train_y's shape: (670,)
test_y's shape: (330,)

MyMLPClassifier(
(linear): Linear(in_features=2, out_features=4, bias=True)
(tanh): Tanh()
(linear): Linear(in_features=4, out_features=2, bias=True)
)
Epoch: 0, loss=0.8484, train_acc=0.4433
Epoch: 1, loss=0.6542, train_acc=0.5881
Epoch: 2, loss=0.6235, train_acc=0.6373
Epoch: 3, loss=0.6097, train_acc=0.6418
Epoch: 4, loss=0.5993, train_acc=0.6493
Epoch: 5, loss=0.5885, train_acc=0.6582
Epoch: 6, loss=0.5757, train_acc=0.6940
Epoch: 7, loss=0.5607, train_acc=0.7687
Epoch: 8, loss=0.5443, train_acc=0.7925
Epoch: 9, loss=0.5278, train_acc=0.8134
Epoch: 10, loss=0.5122, train_acc=0.8328
Epoch: 11, loss=0.4980, train_acc=0.8522
Epoch: 12, loss=0.4854, train_acc=0.8582
Epoch: 13, loss=0.4745, train_acc=0.8612
Epoch: 14, loss=0.4651, train_acc=0.8672
Epoch: 15, loss=0.4570, train_acc=0.8687
Epoch: 16, loss=0.4500, train_acc=0.8746
Epoch: 17, loss=0.4440, train_acc=0.8776
Epoch: 18, loss=0.438

/home/funny/miniconda3/envs/pytorch-env/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


# 3. Binary classification with planar data 
(Use binary cross entroy loss - BCE loss)

In [20]:
def prepare_data():
    X, Y = load_planar_dataset()
    X_tr, X_te, y_tr, y_te = train_test_split(X, Y, test_size=0.33, random_state=42)
    
    print ("train_x's shape: " + str(X_tr.shape))
    print ("test_x's shape: " + str(X_te.shape))
    print ("train_y's shape: " + str(y_tr.shape))
    print ("test_y's shape: " + str(y_te.shape))
    print()
    
    return X_tr, X_te, y_tr, y_te


def prepare_trainer(model):
    optimizer = SGDOptimizer(model, learning_rate=0.1, regularization=0.03, decay_learning_rate=False)
    loss_func = nn.BCELoss()
    return optimizer, loss_func

def build_model(n_in, n_class):
    np.random.seed(101)
    model = MyMLPClassifier(n_input=n_in, hiddens=[4], n_output=1, activation='tanh')   
    return model

In [21]:
def run_my_mlp_for_binary_classification3():
    X_tr, X_te, y_tr, y_te = prepare_data()
    
    n_in, n_class = X_tr.shape[1], 2

    num_epochs = 480
    batch_size = 32

    model = build_model(n_in, n_class)
    optimizer, loss_func = prepare_trainer(model)

    model.info()

    # Training loop
    for epoch in range(num_epochs):
        # Prepare the data loader for training data
        data_loader = prepare_data_loader(X_tr, y_tr, batch_size)
        
        # Initialize counters for tracking training progress
        step = 0
        total_loss, total_correct = 0, 0
        total_sample = 0

        for batch_X, batch_y in data_loader:
            # forward pass: compute logits and loss                     
            batch_logit = model.forward(batch_X)    # output model: logit 
            loss = loss_func.forward(batch_logit, batch_y)

            # backward pass and an optimization step
            optimizer.zero_grad()
            dout = loss_func.backward()
            model.backward(dout)
            optimizer.step()

            # log training progress
            step += 1
            total_loss += loss
            probability = loss_func.probability     # logit --> probability
            batch_yp = (probability >= 0.5).astype(int).flatten()
            total_correct += np.sum(batch_yp == batch_y)
            total_sample += len(batch_y)
        print(f"Epoch: {epoch}, loss={total_loss / total_sample:.4f}, train_acc={total_correct / total_sample:.4f}")

    model.eval()

    logits_test = model.forward(X_te)
    probability = nn.Sigmoid().forward(logits_test)
    ypred = (probability >= 0.5).astype(int)
    print("\n My model: Classification report of test set:\n", classification_report(y_te, ypred))

    
    skmodel = MLPClassifier()
    skmodel.fit(X_tr, y_tr)

     # Print information about the trained scikit-learn MLPClassifier
    print("\nScikit-learn MLPClassifier Info:")
    print("Number of layers:", skmodel.n_layers_)
    print("Number of neurons in each layer:", skmodel.hidden_layer_sizes)
    print("Number of output classes:", skmodel.n_outputs_)
    print("Activation function:", skmodel.activation)
    print("Solver:", skmodel.solver)
    print("Learning rate:", skmodel.learning_rate)
    print("Initial learning rate:", skmodel.learning_rate_init)
    print("Batch size:", skmodel.batch_size)
    print("Maximum number of iterations:", skmodel.max_iter)
    # Add more model-specific information as needed

    ypred = skmodel.predict(X_te)
    print("\n Sklearn model: Classification report of test set:\n", classification_report(y_te, ypred))

In [22]:
model = run_my_mlp_for_binary_classification3()

train_x's shape: (670, 2)
test_x's shape: (330, 2)
train_y's shape: (670,)
test_y's shape: (330,)

MyMLPClassifier(
(linear): Linear(in_features=2, out_features=4, bias=True)
(tanh): Tanh()
(linear): Linear(in_features=4, out_features=1, bias=True)
)
Epoch: 0, loss=0.0203, train_acc=0.6239
Epoch: 1, loss=0.0195, train_acc=0.5940
Epoch: 2, loss=0.0192, train_acc=0.5060
Epoch: 3, loss=0.0190, train_acc=0.5045
Epoch: 4, loss=0.0189, train_acc=0.5239
Epoch: 5, loss=0.0188, train_acc=0.5716
Epoch: 6, loss=0.0188, train_acc=0.5970
Epoch: 7, loss=0.0188, train_acc=0.6060
Epoch: 8, loss=0.0189, train_acc=0.6075
Epoch: 9, loss=0.0189, train_acc=0.6075
Epoch: 10, loss=0.0189, train_acc=0.6075
Epoch: 11, loss=0.0189, train_acc=0.6075
Epoch: 12, loss=0.0189, train_acc=0.6075
Epoch: 13, loss=0.0190, train_acc=0.6060
Epoch: 14, loss=0.0190, train_acc=0.6015
Epoch: 15, loss=0.0190, train_acc=0.6015
Epoch: 16, loss=0.0190, train_acc=0.6000
Epoch: 17, loss=0.0190, train_acc=0.5985
Epoch: 18, loss=0.019

/home/funny/miniconda3/envs/pytorch-env/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
